In [2]:
import re
import json
from krutrim_cloud import KrutrimCloud
from multiprocessing import Pool

client = KrutrimCloud(api_key='QeLB13Ukg0C3lcDTX_bBF')

fall_abstracts_path = 'Fall_Abstracts(2011-2018).txt'
spring_abstracts_path = 'Spring_Abstracts(2011-2018).txt'
output_json_path = 'abstracts_with_5_keywords.json'
checkpoint_path = 'processed_abstracts_checkpoint.json'

def extract_keywords(abstract):
    model_name = "Llama-3.3-70B-Instruct"
    messages = [
        {"role": "system", "content": "You are an expert keyword extractor specialized in materials science and engineering. Your task is to analyze abstracts from the Materials Research Society (MRS) conferences and extract exactly 5 concise and specific keywords or phrases from each abstract. These keywords should: 1. Reflect the core topics, materials, techniques, or methods discussed in the abstract. 2. Be specific to the research focus of the abstract (e.g., 'perovskite solar cells' instead of 'solar cells'). 3. Avoid overly generic terms (e.g., 'materials science' or 'analysis'). 4. Be formatted as a comma-separated list without any additional text or explanation. Your responses should be precise, consistent, and ready for automated analysis. If the abstract is unclear or lacks details, generate the best possible keywords based on the provided information."},
        {
            "role": "user",
            "content": "Abstract: " + abstract + " Extract exactly 5 concise and specific keywords or phrases that capture the core topics, techniques, materials, or methods discussed in this abstract. Ensure the keywords are relevant, specific, and formatted as a comma-separated list without any additional text or explanation. Output format: Keyword1, Keyword2, Keyword3, Keyword4, Keyword5",
        },
    ]
    try:
        response = client.chat.completions.create(model=model_name, messages=messages)
        txt_output_data = response.choices[0].message.content  # type:ignore
        return txt_output_data
    except Exception as exc:
        print(f"Exception: {exc}")
        return None

def process_abstract(line, current_year):
    # Extract country as the last word before "Show Abstract"
    pre_abstract_text = line.split("Show Abstract")[0]
    country_match = re.search(r"(\b\w+)$", pre_abstract_text.strip())
    country = country_match.group(1) if country_match else "Unknown Country"

    # Extract keywords
    keywords = extract_keywords(line)
    if not keywords:
        return None

    # Return processed data
    return {
        "year": current_year,
        "country": country,
        "keywords": [keyword.strip() for keyword in keywords.split(", ")],
    }

def process_abstract_file(file_path, checkpoint):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()

    current_year = None
    processed_data = checkpoint or []
    processed_lines = set(item["line_number"] for item in processed_data)
    # print(processed_lines)
    year_pattern = re.compile(r"(\d{4}) Symposiums")
    results = []

    for line_number, line in enumerate(content):
        if line_number in processed_lines:
            continue

        # Check for year
        year_match = year_pattern.match(line.strip())
        if year_match:
            current_year = int(year_match.group(1))
            continue

        # Check for abstracts
        if "Show Abstract" in line and current_year:
            result = process_abstract(line, current_year)
            if result:
                result["line_number"] = line_number
                processed_data.append(result)
                results.append(result)

            # Save checkpoint periodically
            if len(results) >= 200:
                with open(checkpoint_path, 'w') as checkpoint_file:
                    json.dump(processed_data, checkpoint_file, indent=4)
                results.clear()

    return processed_data

c:\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Python311\Lib\site-packages\traitlets\config\application.py",

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 725, in start
    self.io_loop.start()
  File "c:\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in 

AttributeError: _ARRAY_API not found

In [3]:
try:
    # Load checkpoint if it exists
    try:
        with open(checkpoint_path, 'r') as checkpoint_file:
            checkpoint_data = json.load(checkpoint_file)
    except FileNotFoundError:
        checkpoint_data = []
            # Process files
    # fall_data = process_abstract_file(fall_abstracts_path, checkpoint_data)
    spring_data = process_abstract_file(spring_abstracts_path, checkpoint_data)

    # Combine data and write to JSON
    with open(output_json_path, 'w') as json_file:
        json.dump(spring_data, json_file, indent=4)

    # Clean up checkpoint
    if checkpoint_path.exists():
        checkpoint_path.unlink()

except Exception as e:
    print(f"Error occurred: {e}")

Exception: <HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;cloud&#46;olakrutrim&#46;com&#47;v1&#47;chat&#47;completions" on this server.<P>
Reference&#32;&#35;18&#46;1d69c317&#46;1737875082&#46;52f03d70
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;1d69c317&#46;1737875082&#46;52f03d70</P>
</BODY>
</HTML>
Exception: <HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;cloud&#46;olakrutrim&#46;com&#47;v1&#47;chat&#47;completions" on this server.<P>
Reference&#32;&#35;18&#46;2669c317&#46;1737875082&#46;11779700
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;2669c317&#46;1737875082&#46;11779700</P>
</BODY>
</HTML>
Exception: <HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;cloud&#46;olakrutrim&#46;com

In [37]:
#2024
import re
import json
import html
from krutrim_cloud import KrutrimCloud

client = KrutrimCloud(api_key='QeLB13Ukg0C3lcDTX_bBF')

abstracts_path = './data_scraped/2024-spring.txt'
output_json_path = 'keywords-spring-2024.json'
checkpoint_path = 'processed_checkpoint.json'

def clean_html(text):
    """Remove HTML tags and unescape HTML entities."""
    clean_text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
    return html.unescape(clean_text)  # Convert HTML entities

def extract_abstracts(content):
    """Extract abstracts by capturing everything between 'Article:' occurrences."""
    articles = re.split(r'Article:', content)[1:]  # Split and remove first empty item
    abstracts = [clean_html(article.strip()) for article in articles]
    return abstracts


def extract_keywords_and_country(abstract):
    """Use LLM to extract exactly 5 keywords and the country from the abstract."""
    model_name = "Llama-3.3-70B-Instruct"
    messages = [
        {"role": "system", "content": "You are an expert keyword extractor specialized in materials science and engineering. Your task is to analyze abstracts from the Materials Research Society (MRS) conferences and extract exactly 5 concise and specific keywords or phrases from each abstract. These keywords should: 1. Reflect the core topics, materials, techniques, or methods discussed in the abstract. 2. Be specific to the research focus of the abstract (e.g., 'perovskite solar cells' instead of 'solar cells'). 3. Avoid overly generic terms (e.g., 'materials science' or 'analysis'). 4. Be formatted as a comma-separated list without any additional text or explanation. Your responses should be precise, consistent, and ready for automated analysis. If the abstract is unclear or lacks details, generate the best possible keywords based on the provided information."},
        {"role": "user", "content": f"Abstract: {abstract}\nExtract exactly 5 concise and specific keywords or phrases that capture the core topics, techniques, materials, or methods discussed in this abstract and the country of the university. Format: Keyword1, Keyword2, Keyword3, Keyword4, Keyword5 | Country"},
    ]
    try:
        response = client.chat.completions.create(model=model_name, messages=messages)
        txt_output_data = response.choices[0].message.content  # type:ignore
        keywords, country = txt_output_data.split('|')
        return [kw.strip() for kw in keywords.split(',')], country.strip()
    except Exception as exc:
        print(f"Exception: {exc}")
        return None, "Unknown"
    


def process_abstract_file(file_path, checkpoint):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    abstracts = extract_abstracts(content)
    processed_data = checkpoint or []
    processed_lines = set(item["line_number"] for item in processed_data)
    results = []
    
    for line_number, abstract in enumerate(abstracts):
        if line_number in processed_lines:
            continue
        keywords, country = extract_keywords_and_country(abstract)
        if keywords:
            result = {"year": 2024, "abstract": abstract, "keywords": keywords, "country": country, "line_number": line_number}
            processed_data.append(result)
            results.append(result)
        
        # Save checkpoint periodically
        if len(results) >= 200:
            with open(checkpoint_path, 'w') as checkpoint_file:
                json.dump(processed_data, checkpoint_file, indent=4)
            results.clear()
    
    return processed_data

In [38]:
try:
    # Load checkpoint if it exists
    try:
        with open(checkpoint_path, 'r') as checkpoint_file:
            checkpoint_data = json.load(checkpoint_file)
    except FileNotFoundError:
        checkpoint_data = []

    data = process_abstract_file(abstracts_path, checkpoint_data)

    # Save extracted abstracts, keywords, and countries to JSON
    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    # Clean up checkpoint
    if checkpoint_path.exists():
        checkpoint_path.unlink()

except Exception as e:
    print(f"Error occurred: {e}")


KeyboardInterrupt: 

In [2]:
#21-23
import re
import json
from krutrim_cloud import KrutrimCloud

client = KrutrimCloud(api_key='QeLB13Ukg0C3lcDTX_bBF')

abstracts_path = './data/2020-spring-fall-final.txt'
output_json_path = 'keywords-spring-fall-2020.json'
checkpoint_path = 'processed_checkpoint.json'


def extract_keywords_and_country(abstract):
    """Use LLM to extract exactly 5 keywords and the country from the abstract."""
    model_name = "Llama-3.3-70B-Instruct"
    messages = [
        {"role": "system", "content": "You are an expert keyword extractor specialized in materials science and engineering. Your task is to analyze abstracts from the Materials Research Society (MRS) conferences and extract exactly 5 concise and specific keywords or phrases from each abstract. These keywords should: 1. Reflect the core topics, materials, techniques, or methods discussed in the abstract. 2. Be specific to the research focus of the abstract (e.g., 'perovskite solar cells' instead of 'solar cells'). 3. Avoid overly generic terms (e.g., 'materials science' or 'analysis'). 4. Be formatted as a comma-separated list without any additional text or explanation. Your responses should be precise, consistent, and ready for automated analysis. If the abstract is unclear or lacks details, generate the best possible keywords based on the provided information."},
        {"role": "user", "content": f"Abstract: {abstract}\nExtract exactly 5 concise and specific keywords or phrases that capture the core topics, techniques, materials, or methods discussed in this abstract and the country of the university. Format: Keyword1, Keyword2, Keyword3, Keyword4, Keyword5 | Country"},
    ]
    try:
        response = client.chat.completions.create(model=model_name, messages=messages)
        txt_output_data = response.choices[0].message.content  # type:ignore
        keywords, country = txt_output_data.split('|')
        return [kw.strip() for kw in keywords.split(',')], country.strip()
    except Exception as exc:
        print(f"Exception: {exc}")
        return None, "Unknown"

def process_abstract_file(file_path, checkpoint):
    abstracts = []
    with open(file_path, 'r', encoding='utf-8') as file:
        abstracts = [abstract for abstract in file.readlines() if abstract.strip()]
    # print(abstracts[0])
    processed_data = checkpoint or []
    processed_lines = set(item["line_number"] for item in processed_data)
    results = []
    
    for line_number, abstract in enumerate(abstracts):
        if line_number in processed_lines:
            continue
        # print(abstract)
        keywords, country = extract_keywords_and_country(abstract)
        if keywords:
            result = {"year": 2021, "abstract": abstract, "keywords": keywords, "country": country, "line_number": line_number}
            processed_data.append(result)
            results.append(result)
        
        # Save checkpoint periodically
        if len(results) >= 200:
            with open(checkpoint_path, 'w') as checkpoint_file:
                json.dump(processed_data, checkpoint_file, indent=4)
            results.clear()
    
    return processed_data

In [3]:
try:
    # Load checkpoint if it exists
    try:
        with open(checkpoint_path, 'r') as checkpoint_file:
            checkpoint_data = json.load(checkpoint_file)
    except FileNotFoundError:
        checkpoint_data = []

    data = process_abstract_file(abstracts_path, checkpoint_data)

    # Save extracted abstracts, keywords, and countries to JSON
    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    # Clean up checkpoint
    import os
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)

except Exception as e:
    print(f"Error occurred: {e}")

Exception: not enough values to unpack (expected 2, got 1)
Exception: <HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;cloud&#46;olakrutrim&#46;com&#47;v1&#47;chat&#47;completions" on this server.<P>
Reference&#32;&#35;18&#46;c7014017&#46;1738856453&#46;1bc3b6c3
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;c7014017&#46;1738856453&#46;1bc3b6c3</P>
</BODY>
</HTML>
Exception: not enough values to unpack (expected 2, got 1)
Exception: not enough values to unpack (expected 2, got 1)
Exception: too many values to unpack (expected 2)
Exception: not enough values to unpack (expected 2, got 1)
Exception: not enough values to unpack (expected 2, got 1)
Exception: not enough values to unpack (expected 2, got 1)
Exception: not enough values to unpack (expected 2, got 1)
Exception: too many values to unpack (expected 2)
Exception: not enough values to unpack (expected 2, got 1)
Exception: not e

In [1]:
import re
import json
from krutrim_cloud import KrutrimCloud

# Initialize API client
client = KrutrimCloud(api_key='QeLB13Ukg0C3lcDTX_bBF')

# File paths
fall_abstracts_path = 'Fall_Abstracts(2011-2018).txt'
spring_abstracts_path = 'Spring_Abstracts(2011-2018).txt'
output_json_path = 'abstracts_with_5_keywords.json'
checkpoint_path = 'processed_abstracts_checkpoint.json'

# Function to extract keywords
def extract_keywords(abstract):
    model_name = "Llama-3.3-70B-Instruct"
    messages = [
        {"role": "system", "content": "You are an expert keyword extractor specialized in materials science and engineering. Your task is to analyze abstracts from the Materials Research Society (MRS) conferences and extract exactly 5 concise and specific keywords or phrases from each abstract. These keywords should: 1. Reflect the core topics, materials, techniques, or methods discussed in the abstract. 2. Be specific to the research focus of the abstract (e.g., 'perovskite solar cells' instead of 'solar cells'). 3. Avoid overly generic terms (e.g., 'materials science' or 'analysis'). 4. Be formatted as a comma-separated list without any additional text or explanation."},
        {
            "role": "user",
            "content": "Abstract: " + abstract + " Extract exactly 5 concise and specific keywords or phrases that capture the core topics, techniques, materials, or methods discussed in this abstract. Ensure the keywords are relevant, specific, and formatted as a comma-separated list without any additional text or explanation. Output format: Keyword1, Keyword2, Keyword3, Keyword4, Keyword5",
        },
    ]
    try:
        response = client.chat.completions.create(model=model_name, messages=messages)
        return response.choices[0].message.content  # type: ignore
    except Exception as exc:
        print(f"API Error: {exc}")
        return None

# Process a single abstract
def process_abstract(line, current_year):
    try:
        # Extract country as the last word before "Show Abstract"
        pre_abstract_text = line.split("Show Abstract")[0]
        country_match = re.search(r"(\b\w+)$", pre_abstract_text.strip())
        country = country_match.group(1) if country_match else "Unknown Country"

        # Extract keywords
        keywords = extract_keywords(line)
        if not keywords:
            print(f"Failed to extract keywords for line: {line}")
            return None

        return {
            "year": current_year,
            "country": country,
            "keywords": [keyword.strip() for keyword in keywords.split(", ")],
        }
    except Exception as e:
        print(f"Error processing line: {line}\n{e}")
        return None

# Process the file
def process_abstract_file(file_path, checkpoint=None):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()

    current_year = None
    processed_data = checkpoint or []
    processed_lines = set(item["line_number"] for item in processed_data)

    year_pattern = re.compile(r"(\d{4}) Symposiums")

    for line_number, line in enumerate(content):
        if line_number in processed_lines:
            continue

        # Detect the year
        year_match = year_pattern.match(line.strip())
        if year_match:
            current_year = int(year_match.group(1))
            continue

        # Process abstracts
        if "Show Abstract" in line and current_year:
            print(f"Processing line {line_number} for year {current_year}")
            result = process_abstract(line, current_year)
            if result:
                result["line_number"] = line_number
                processed_data.append(result)

            # Save checkpoint
            if len(processed_data) % 100 == 0:
                with open(checkpoint_path, 'w') as checkpoint_file:
                    json.dump(processed_data, checkpoint_file, indent=4)

    return processed_data

# Load checkpoint
try:
    with open(checkpoint_path, 'r') as checkpoint_file:
        checkpoint_data = json.load(checkpoint_file)
except FileNotFoundError:
    checkpoint_data = []

# Process files
fall_data = process_abstract_file(fall_abstracts_path, checkpoint_data)
spring_data = process_abstract_file(spring_abstracts_path, fall_data)

# Save final results
with open(output_json_path, 'w') as json_file:
    json.dump(spring_data, json_file, indent=4)

print("Processing complete.")


c:\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Python311\Lib\site-packages\traitlets\config\application.py",

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Python311\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 725, in start
    self.io_loop.start()
  File "c:\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 195, in 

AttributeError: _ARRAY_API not found

Processing line 1596 for year 2011
API Error: <HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;cloud&#46;olakrutrim&#46;com&#47;v1&#47;chat&#47;completions" on this server.<P>
Reference&#32;&#35;18&#46;df014017&#46;1737648498&#46;99778680
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;df014017&#46;1737648498&#46;99778680</P>
</BODY>
</HTML>
Failed to extract keywords for line: 9:00 PM - F4.9 Large Scale Synthesis and Characterization of Micro/Nano-Scale LiFePO4/C Composite Cathode Materials. Xiangcheng Sun 1 , Kai Sun 2 , Bo Cui 1 1 Electrical and Computer Engineering, University of Waterloo, Waterloo, Ontario, Canada, 2 Department of Materials Science and Engineering, University of Michigan, Ann Arbor, Michigan, United StatesShow AbstractSince the pioneering work of Goodenough in 1997 [1], because of its suitable operating voltage and high theoretical capacity (170 mAh/g) as well as

KeyboardInterrupt: 

# process clusters

In [4]:
import os
import json
from krutrim_cloud import KrutrimCloud

# Initialize API client
client = KrutrimCloud(api_key='QeLB13Ukg0C3lcDTX_bBF')

# Directory containing clustered keyword JSON files
INPUT_DIR = "./clustering_results_new"  # Adjust if needed
OUTPUT_DIR = "./cluster_labels"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Years you have cluster files for
YEARS = list(range(2011, 2025))  # 2011 to 2024

# LLM model config
MODEL_NAME = "Llama-3.3-70B-Instruct"

# Function to get representative label from LLM
def label_cluster_with_llm(cluster_keywords):
    keywords_text = ", ".join(cluster_keywords)
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in materials science. Your task is to assign a single representative keyword or phrase "
                "to a given list of related materials science research keywords. Choose a concise, specific phrase that "
                "accurately summarizes the entire cluster. If the keywords are too unrelated or vary widely, return only: None.\n\n"
                "Rules:\n"
                "1. Choose a single short phrase or keyword (no more than 4-5 words).\n"
                "2. Make it specific and relevant (e.g., 'Mott-Schottky analysis', not 'materials characterization').\n"
                "3. Output only the label or 'None' with no additional explanation."
            )
        },
        {
            "role": "user",
            "content": f"Cluster: {keywords_text}\n\nOutput a single label or 'None':"
        },
    ]
    try:
        response = client.chat.completions.create(model=MODEL_NAME, messages=messages)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ API Error: {e}")
        return "None"

# Process each year's cluster file
for year in YEARS:
    cluster_file = os.path.join(INPUT_DIR, f"{year}_fastclust_dt1.json")  # Adjust filename format if needed
    if not os.path.exists(cluster_file):
        print(f"❌ Missing: {cluster_file}")
        continue

    print(f"\n📦 Processing clusters from: {year}")
    with open(cluster_file, 'r', encoding='utf-8') as f:
        clusters = json.load(f)

    labeled_clusters = {}
    for cluster_id, keywords in clusters.items():
        # print(f"→ Cluster {cluster_id} ({len(keywords)} keywords)")
        label = label_cluster_with_llm(keywords)
        # print(f"   🏷️ Label: {label}")
        labeled_clusters[cluster_id] = label

    # Save labeled output
    output_path = os.path.join(OUTPUT_DIR, f"{year}_labeled_clusters.json")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(labeled_clusters, f, indent=2, ensure_ascii=False)

print("\n✅ Done labeling all clusters.")



📦 Processing clusters from: 2011

📦 Processing clusters from: 2012

📦 Processing clusters from: 2013

📦 Processing clusters from: 2014

📦 Processing clusters from: 2015

📦 Processing clusters from: 2016

📦 Processing clusters from: 2017

📦 Processing clusters from: 2018

📦 Processing clusters from: 2019

📦 Processing clusters from: 2020

📦 Processing clusters from: 2021

📦 Processing clusters from: 2022

📦 Processing clusters from: 2023

📦 Processing clusters from: 2024

✅ Done labeling all clusters.


In [9]:
import os
import json
from collections import defaultdict

# Directory where the per-year JSONs are stored
INPUT_DIR1 = "./clustering_results_new"
INPUT_DIR2 = "./cluster_labels"  # Adjust if needed
OUTPUT_JSON = "keyword_timeline.json"

# Years to process
YEARS = list(range(2011, 2025))

# Output dictionary
label_year_counts = defaultdict(lambda: defaultdict(int))

for year in YEARS:
    cluster_path = os.path.join(INPUT_DIR1, f"{year}_fastclust_dt1.json")
    label_path = os.path.join(INPUT_DIR2, f"{year}_labeled_clusters.json")

    if not os.path.exists(cluster_path) or not os.path.exists(label_path):
        print(f"Skipping {year} (files not found)")
        continue

    with open(cluster_path, "r", encoding='utf-8') as f:
        cluster_data = json.load(f)

    with open(label_path, "r", encoding='utf-8') as f:
        label_data = json.load(f)

    for cluster_id, keywords in cluster_data.items():
        label = label_data.get(cluster_id)
        if label is not None:
            label_year_counts[label][str(year)] += len(keywords)

# Convert defaultdict to normal dict for JSON output
final_result = {label: dict(years) for label, years in label_year_counts.items()}

# Save to JSON
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(final_result, f, indent=2, ensure_ascii=False)

print(f"✅ Saved keyword label year-count data to {OUTPUT_JSON}")


✅ Saved keyword label year-count data to keyword_timeline.json


In [6]:
import os
import json
from collections import defaultdict
from krutrim_cloud import KrutrimCloud

# === Config ===
THRESHOLDS = [0.5, 1, 1.5]
OUTPUT_DIR = "./labeled_cluster_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === API Client ===
client = KrutrimCloud(api_key='QeLB13Ukg0C3lcDTX_bBF')
MODEL_NAME = "Llama-3.3-70B-Instruct"

def label_cluster_with_llm(cluster_keywords):
    keywords_text = ", ".join(cluster_keywords)
    messages = [
        {
            "role": "system",
            "content": (
                "You are an expert in materials science. Your task is to assign a single representative keyword or phrase "
                "to a given list of related materials science research keywords. Choose a concise, specific phrase that "
                "accurately summarizes the entire cluster. If the keywords are too unrelated or vary widely, return only: None.\n\n"
                "Rules:\n"
                "1. Choose a single short phrase or keyword (no more than 4-5 words).\n"
                "2. Make it specific and relevant (e.g., 'Mott-Schottky analysis', not 'materials characterization').\n"
                "3. Output only the label or 'None' with no additional explanation."
            )
        },
        {
            "role": "user",
            "content": f"Cluster: {keywords_text}\n\nOutput a single label or 'None':"
        },
    ]
    try:
        response = client.chat.completions.create(model=MODEL_NAME, messages=messages)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ API Error: {e}")
        return "None"

# === Process each threshold file ===
for thresh in THRESHOLDS:
    input_file = f"clusters_dt{thresh}_with_year_counts.json"
    with open(input_file, 'r', encoding='utf-8') as f:
        clusters = json.load(f)

    print(f"\n🔍 Processing {input_file}...")

    labeled_output = defaultdict(lambda: defaultdict(int))

    for cluster_id, cluster_data in clusters.items():
        keywords = cluster_data["keywords"]
        years = cluster_data["years"]

        label = label_cluster_with_llm(keywords)
        if label == "None":
            continue

        for year, count in years.items():
            labeled_output[label][str(year)] += count

    # Sort inner dicts by year
    final_output = {
        label: dict(sorted(year_data.items()))
        for label, year_data in labeled_output.items()
    }

    # Save
    output_file = os.path.join(OUTPUT_DIR, f"labeled_clusters_dt{thresh}.json")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved labeled results to: {output_file}")



🔍 Processing clusters_dt0.5_with_year_counts.json...
✅ Saved labeled results to: ./labeled_cluster_results\labeled_clusters_dt0.5.json

🔍 Processing clusters_dt1_with_year_counts.json...
✅ Saved labeled results to: ./labeled_cluster_results\labeled_clusters_dt1.json

🔍 Processing clusters_dt1.5_with_year_counts.json...
✅ Saved labeled results to: ./labeled_cluster_results\labeled_clusters_dt1.5.json


In [2]:
# === save_embeddings.py ===
import os
import json
import numpy as np
from sentence_transformers import SentenceTransformer

FILE_MAP = {
    "Fine-grained (0.5 threshold, ~9000)": "labeled_clusters_dt0.5.json",
    "Moderate (1.0 threshold, ~3000)": "labeled_clusters_dt1.json",
    "Broad (1.5 threshold, ~1600)": "labeled_clusters_dt1.5.json"
}
DATA_DIR = "./labeled_cluster_results"
SAVE_DIR = "./precomputed_embeddings"

os.makedirs(SAVE_DIR, exist_ok=True)

model = SentenceTransformer("all-MiniLM-L6-v2")

for label, file in FILE_MAP.items():
    file_path = os.path.join(DATA_DIR, file)
    with open(file_path, "r", encoding="utf-8") as f:
        cluster = json.load(f)
        keywords = list(cluster.keys())

    embeddings = model.encode(keywords, show_progress_bar=True)

    # Save with label key in filename
    base_name = label.split("(")[0].strip().replace(" ", "_").lower()
    np.save(os.path.join(SAVE_DIR, f"{base_name}_embeddings.npy"), embeddings)
    with open(os.path.join(SAVE_DIR, f"{base_name}_keywords.json"), "w") as f:
        json.dump(keywords, f)

    print(f"Saved: {label}")


Batches:   0%|          | 0/212 [00:00<?, ?it/s]

Saved: Fine-grained (0.5 threshold, ~9000)


Batches:   0%|          | 0/111 [00:00<?, ?it/s]

Saved: Moderate (1.0 threshold, ~3000)


Batches:   0%|          | 0/50 [00:00<?, ?it/s]

Saved: Broad (1.5 threshold, ~1600)
